In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
covid_19_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv', index_col = 'ObservationDate')
covid_19_data

In [ ]:
covid_19_data['Country/Region'].value_counts()

Data from many countries, make filter per Rus

In [ ]:

data_from_rus = covid_19_data[covid_19_data['Country/Region'].isin(['Russia'])]
data_from_rus

First rows contains total info jf all Russia. Try to group by date

In [ ]:
data = data_from_rus.groupby('ObservationDate').sum()['Confirmed']
data

485 days observ

In [ ]:
# xxx = pd.DataFrame([['05/29/2021',2,'moscow'],
#                     ['05/29/2021',20, 'moscow'],
#                     ['01/31/2020',4,'NaN'], 
#                     ['02/02/2020',5,'NaN']],
#                    columns=['ObservationDate','Confirmed','Province/State'])
# xxx.groupby('ObservationDate').sum()['Confirmed']

let's sort by date

In [ ]:
data.sort_values()

and make numpy 1D array

In [ ]:
df = np.array(data.sort_values())
df[:10]

morbidity chart

In [ ]:
data_from_rus['Confirmed'].plot( figsize = (16,5))

plt.legend()
plt.grid('On')
plt.show()

normalize data

In [ ]:
scaler = StandardScaler()
dataset = scaler.fit_transform(df.reshape(-1, 1))
dataset[0][:10]

train test split

In [ ]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('train shape ', train.shape)
print('test shape ', test.shape)

Длина теста 161, а предикт мы выдаем на 151 (-10 для look back)

In [ ]:
# Функция создания датасета
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# кол-во следующих значений
look_back = 40 

# train test split
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

print('trainX ',trainX.shape)
print('testX ',testX.shape)

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
trainX.shape

In [ ]:
batch_size = 1

model = Sequential()
model.add(LSTM(128, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='sgd')
for i in range(60):
	model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()

In [ ]:
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(testX, batch_size=batch_size)

In [ ]:
# trainPredict

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
import math
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
testPredict[-9:-1]

In [ ]:
realdata = data.sort_values()
realdata[-9:-1]

In [ ]:
# realdata.index[-150:]
predicted_data = pd.DataFrame(testPredict, index=realdata.index[-120:])
predicted_data

In [ ]:
plt.figure(figsize=(14,5))

# plt.plot(realdata[-150:], )
plt.plot(realdata, label='real data')
plt.plot(predicted_data, label='predicted')
plt.xticks([realdata.index[0], predicted_data.index[0] , realdata.index[-1]])
plt.vlines(predicted_data.index[0], realdata.min(), realdata.max(), color = 'r', label='split')
# plt.grid('on')
plt.legend()
plt.xlabel('дни')
plt.show()

In [ ]:
testPredict.shape

Попробую сделать прогноз на будущее

In [ ]:
from datetime import datetime, timedelta
# realdata.index[-1]
# future_predicts = np.array([])
# future_predicts = np.append(future_predicts, 1)
# future_predicts = np.append(future_predicts, 2)
# future_predicts
# future_predicts = pd.DataFrame(columns = ['ObservationDate', 'Confirmed'])#, index= 'ObservationDate')
# future_predicts.append({'ObservationDate':'25/01/2021','Confirmed':'150'}, ignore_index=True)

# predicted_data = pd.DataFrame(testPredict, index=realdata.index[-151:-1])
# predicted_data
# dt = datetime.strptime(realdata.index[-1], '%m/%d/%Y')
# first_date = np.array([dt + timedelta(days=1)])
# first_date[0].strftime('%m/%d/%Y')
# date_index = pd.DataFrame([realdata.index[-1]], columns=['ObservationDate'])
# date_index


In [ ]:
def future_predict(model, last_look_back_days, future_days=50):
    future_predicts = np.array([])
    dt = datetime.strptime(realdata.index[-1], '%m/%d/%Y')
    dates = np.array([])
    
    for day in range(future_days):
        dataset = scaler.fit_transform(last_look_back_days.reshape(-1, 1))
#         dataset = scaler.fit_transform(last_look_back_days)
        testX, testY = create_dataset(dataset, look_back=len(last_look_back_days)-2)
        testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
        new_predict = model.predict(testX, batch_size=batch_size)
        new_predict = scaler.inverse_transform(new_predict)
        future_predicts = np.append(future_predicts, new_predict)
        
        dates = np.append(dates, [dt + timedelta(days=day)])
        dates[day].strftime('%m/%d/%Y')
        
        model.reset_states()
        last_look_back_days = np.delete(last_look_back_days, 0)
        last_look_back_days = np.append(last_look_back_days, new_predict)
#         new_day_confirmed = testX[-1]
    predicts = pd.DataFrame(future_predicts, index=dates)
    predicts
    return predicts
        

In [ ]:
predicts = future_predict(model, predicted_data.values[-100:], future_days=100)
predicts

In [ ]:
# # np.append(realdata.index, str(predicts.index), )
# # predicts.index
# realdata.index
# # datetime.strptime(realdata.index, '%m/%d/%Y')
# for date in realdata.index:
    

In [ ]:
plt.figure(figsize=(14,5))

# plt.plot(realdata, label='real data')
plt.plot(predicts, label='predicts')
# plt.xticks([realdata.index[0], predicted_data.index[0] , realdata.index[-1]])
# plt.vlines(predicted_data.index[0], realdata.min(), realdata.max(), color = 'r')
# plt.grid('on')
plt.legend()
plt.xlabel('дни')
plt.show()